In [1]:
import json
import os
import pandas as pd
from fractions import Fraction
from music21 import converter, chord, note, meter, tempo, stream, harmony, roman

In [2]:
DATA_PATH = os.path.join('data', 'MidiCaps')

In [3]:
def quantize_rational(offset, quarterLength):
    onset = Fraction(offset).limit_denominator(32)
    duration = Fraction(quarterLength).limit_denominator(32)
    return {
        "onset": {"numerator": onset.numerator, "denominator": onset.denominator},
        "duration": {"numerator": duration.numerator, "denominator": duration.denominator}
    }

def extract_notes(part, key_signature, measure_offset=0):
    measures = []
    for i, m in enumerate(part.getElementsByClass(stream.Measure), start=1):
        measure_notes = []
        for el in m.notesAndRests:
            if isinstance(el, note.Note):
                scale_deg = key_signature.getScale().getScaleDegreeFromPitch(el.pitch)
                note_data = {
                    "pitch": el.pitch.midi,
                    "pitch_class": el.pitch.name,
                    "scale_degree": scale_deg,
                    **quantize_rational(el.offset, el.quarterLength),
                    "velocity": 100
                }
                measure_notes.append(note_data)
        if measure_notes:
            measures.append({
                "measure_number": i + measure_offset,
                "notes": measure_notes
            })
    return measures

def get_function_from_roman(rn_figure: str) -> str:
    rn = rn_figure.lower()

    if rn.startswith("it6") or rn.startswith("ger") or rn.startswith("fr") or rn.startswith("sw"):
        return "augmented_sixth"
    if rn.startswith("bii"):
        return "neapolitan"
    if "/" in rn:
        return "secondary_dominant"
    if rn.startswith("v") or rn.startswith("vii"):
        return "dominant"
    if rn.startswith("ii") or rn.startswith("iv"):
        return "subdominant"
    if rn.startswith("i") or rn.startswith("iii") or rn.startswith("vi"):
        return "tonic"
    if rn.startswith("bvi") or rn.startswith("bvii"):
        return "modal_interchange"
    if rn.startswith("#vii") or rn.startswith("#vi") or rn.startswith("#vio"):
        return "chromatic"

    return "other"

def estimate_voicing_octave(chord_obj, root_octave, min_octave=2, max_octave=6):
    if not chord_obj.notes:
        return None
    avg_pitch = sum(p.midi for p in chord_obj.pitches) / len(chord_obj.pitches)
    estimated_octave = int(avg_pitch // 12) - 1  # MIDI pitch 60 = C4 => octave 4
    if estimated_octave < min_octave or estimated_octave > max_octave:
        return root_octave  # invalid range, fallback
    return estimated_octave

def get_scale_degrees(chord, key_signature):
    scale_degrees = [
        (p.diatonicNoteNum - key_signature.tonic.diatonicNoteNum + 1) % 7 or 7
        for p in chord.pitches
    ]
    return scale_degrees

def extract_harmony_chords(score, key_signature, root_octave=4):
    harmony_data = []

    for measure in score.parts[0].getElementsByClass('Measure'):
        for element in measure.notesAndRests:
            if isinstance(element, chord.Chord):
                c = element
                try:
                    rn = roman.romanNumeralFromChord(c, key_signature)
                except Exception:
                    continue

                voicing_oct = estimate_voicing_octave(c, root_octave)

                harmony_block = {
                    "measure_number": measure.measureNumber,
                    "chord": c.pitchedCommonName,
                    "harmony": {
                        "function": get_function_from_roman(rn.figure),
                        "roman": rn.figure,
                        "scale_degrees": get_scale_degrees(c, key_signature),
                        "voicing_octave": voicing_oct
                    }
                }
                harmony_data.append(harmony_block)
                break  # assume one harmony per measure
    return harmony_data

def midi_to_json_schema(metadata, midi_path):
    score = converter.parse(midi_path)
    key_sig = metadata['key']

    json_data = {
        "metadata": {
            "title": "Converted MIDI",
            "bpm": metadata['tempo'],
            "time_signature": metadata['time_signature'],
            "key_signature": metadata['key'],
            "root_octave": 4
        },
        "tracks": []
    }

    for i, part in enumerate(score.parts):
        inst = part.getInstrument(returnDefault=True)
        track_data = {
            "name": inst.partName or f"track_{i}",
            "instrument": inst.instrumentName or "piano",
            "measures": extract_notes(part, key_sig)
        }
        json_data["tracks"].append(track_data)

    harmony_track = {
        "name": "harmony",
        "instrument": "piano",
        "measures": extract_harmony_chords(score, key_sig)
    }
    json_data["tracks"].append(harmony_track)

    return json_data


In [4]:
df = pd.read_json('data/MidiCaps/train_filtered.json', lines=True)

In [5]:
df.head()

,location,caption,genre,genre_prob,mood,mood_prob,key,time_signature,tempo,tempo_word,...,instrument_summary,instrument_numbers_sorted,all_chords,all_chords_timestamps,test_set,num_tracks,num_time_signatures,num_instruments,instruments,initial_silence
0,lmd_full/1/17655598958db48a34cd882f81402568.mid,A short electronic ambient song featuring a pi...,"[electronic, ambient]","[0.30310000000000004, 0.2369]","[film, energetic, melodic, epic, dark]","[0.11620000000000001, 0.11080000000000001, 0.1...",E major,4/4,170,Presto,...,[Piano],[0],"[B, Emaj7, C#m7, B7, A, B, Emaj7, C#m7, B7, A,...","[0.464399092, 1.30031746, 2.972154195, 3.71519...",False,1,1,1,[Piano],0
1,lmd_full/1/180b3a492c1c1a46d005e762d62b9aa4.mid,A cheerful pop song with a touch of electronic...,"[pop, electronic]","[0.4238, 0.2548]","[happy, love, melodic, christmas, motivational]","[0.1521, 0.09870000000000001, 0.0935, 0.0737, ...",D major,4/4,96,Moderate tempo,...,"[Piano, Drums]","[0, 0, 0, 128, 0]","[D, A7, D, Em, A, D, Em7, A7, D, Em7, A7, D, G...","[0.464399092, 7.616145124, 9.938140589, 12.353...",False,5,1,5,"[bass, Piano, guitar, Percussion, strings]",3
2,lmd_full/1/1f94f64f72af98fd92f206293b281f6f.mid,A classical piece that could be part of a film...,"[classical, soundtrack]","[0.6144000000000001, 0.225]","[film, relaxing, emotional, documentary, drama]","[0.16640000000000002, 0.1048, 0.0655, 0.0621, ...",C# major,4/4,74,Andante,...,[Recorder],[74],"[C#, Fm, Ab, C#, Fm, Bbm, C, Bbm, B, Bb, Ab, F...","[0.464399092, 1.764716553, 9.752380952, 11.609...",False,2,1,1,[Recorder],0
3,lmd_full/1/112139e8b17be9cae1bd14df56e52f14.mid,A short fragment of energetic electronic and c...,"[electronic, classical]","[0.32370000000000004, 0.30610000000000004]","[energetic, film, dark, epic, action]","[0.1081, 0.1009, 0.0961, 0.0916, 0.0896]",F major,4/4,136,Allegro,...,[Piano],"[0, 0]","[A, F/A]","[0.464399092, 1.671836734]",False,2,1,2,"[intro bass (MIDI), Piano]",0
4,lmd_full/1/118dc7024ff75b0bc860f7033dc24982.mid,A very short fragment of energetic electronic ...,"[electronic, pop]","[0.2863, 0.149]","[energetic, happy, melodic, dark, film]","[0.17250000000000001, 0.1206, 0.1159, 0.1044, ...",C major,4/4,150,Fast,...,[Piano],[0],"[F, Fm, E, Em]","[0.464399092, 2.229115646, 3.343673469, 5.5727...",False,1,1,1,[Piano],0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18587 entries, 0 to 18586
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   location                   18587 non-null  object
 1   caption                    18587 non-null  object
 2   genre                      18587 non-null  object
 3   genre_prob                 18587 non-null  object
 4   mood                       18587 non-null  object
 5   mood_prob                  18587 non-null  object
 6   key                        18587 non-null  object
 7   time_signature             18587 non-null  object
 8   tempo                      18587 non-null  int64 
 9   tempo_word                 18587 non-null  object
 10  duration                   18587 non-null  int64 
 11  duration_word              18587 non-null  object
 12  chord_summary              18587 non-null  object
 13  chord_summary_occurence    18587 non-null  int64 
 14  instru

In [6]:
samples = df.sample(10)

In [33]:
#midi_file = os.path.join(DATA_PATH, samples.iloc[0]['location'])
midi_file = os.path.join(DATA_PATH, 'lmd_full/f/f5b01b4a1acba99e38b06f32e80aa4c8.mid')
schema = midi_to_json_schema(midi_file)

# with open(os.path.join('data', 'output.json'), "w") as f:
#     json.dump(schema, f, indent=2)


In [ ]:
{
  "time_signature": "4/4",
  "bpm": 120,
  "key_signature": "C",
  "root_octave": 4,
  "harmony_rhythm_pattern": [
    { "onset": "0/4", "duration": "1/4" },
    { "onset": "2/4", "duration": "1/4" }
  ],
  "tracks": [
    {
      "name": "melody",
      "instrument": "piano",
      "measures": [
        {
          "measure_number": 1,
          "notes": [
            {
              "pitch": 64,
              "scale_degree": 3,
              "pitch_class": "E",
              "duration": "1/4",
              "onset": "0/4",
              "velocity": 90
            },
            {
              "pitch": 67,
              "scale_degree": 5,
              "pitch_class": "G",
              "duration": "1/4",
              "onset": "1/4",
              "velocity": 85
            }
          ]
        }
      ]
    },
    {
      "name": "lead",
      "instrument": "violin",
      "measures": [
        {
          "measure_number": 1,
          "notes": [
            {
              "pitch": 64,
              "scale_degree": 3,
              "pitch_class": "E",
              "duration": "1/4",
              "onset": "0/4",
              "velocity": 90
            },
            {
              "pitch": 67,
              "scale_degree": 5,
              "pitch_class": "G",
              "duration": "1/4",
              "onset": "1/4",
              "velocity": 85
            }
          ]
        }
      ]
    },
    {
      "name": "harmony",
      "instrument": "pad",
      "measures": [
        {
          "measure_number": 1,
          "harmonies": [
            {
              "onset": "0/4",
              "duration": "2/4",
              "chord": "C",
              "harmony": {
                "function": "tonic",
                "roman": "I",
                "scale_degrees": [1, 3, 5],
                "voicing_octave": 3
              }
            },
            {
              "onset": "2/4",
              "duration": "2/4",
              "chord": "G7",
              "harmony": {
                "function": "dominant",
                "roman": "V7",
                "scale_degrees": [1, 3, 5, "flat7"],
                "voicing_octave": 3
              }
            }
          ]
        }
      ]
    }
  ]
}


In [ ]:
[
  {
    "measure": 1,
    "chords": [
      { "chord": "B", "onset": "0/1", "duration": "1/2" },
      { "chord": "Emaj7", "onset": "1/2", "duration": "1/2" }
    ]
  },
  {
    "measure": 2,
    "chords": [
      { "chord": "C#m7", "onset": "0/1", "duration": "1/2" },
      { "chord": "B7", "onset": "1/2", "duration": "1/2" }
    ]
  },
  {
    "measure": 3,
    "chords": [
      { "chord": "A", "onset": "0/1", "duration": "1/1" }
    ]
  },
  {
    "measure": 4,
    "chords": [
      { "chord": "B", "onset": "0/1", "duration": "1/1" }
    ]
  },
  {
    "measure": 5,
    "chords": [
      { "chord": "Emaj7", "onset": "0/1", "duration": "1/1" }
    ]
  }
]

In [34]:
schema

{'metadata': {'title': 'Converted MIDI',
  'composer': 'Unknown',
  'bpm': 190,
  'time_signature': '4/4',
  'key_signature': 'g# minor',
  'root_octave': 4},
 'tracks': [{'name': 'track_0',
   'instrument': 'piano',
   'measures': [{'measure_number': 11,
     'notes': [{'pitch': 73,
       'pitch_class': 'C#',
       'scale_degree': 2,
       'onset': {'numerator': 0, 'denominator': 1},
       'duration': {'numerator': 1, 'denominator': 2},
       'velocity': 100}]},
    {'measure_number': 15,
     'notes': [{'pitch': 73,
       'pitch_class': 'C#',
       'scale_degree': 2,
       'onset': {'numerator': 0, 'denominator': 1},
       'duration': {'numerator': 1, 'denominator': 2},
       'velocity': 100}]},
    {'measure_number': 25,
     'notes': [{'pitch': 63,
       'pitch_class': 'E-',
       'scale_degree': None,
       'onset': {'numerator': 0, 'denominator': 1},
       'duration': {'numerator': 4, 'denominator': 1},
       'velocity': 100}]},
    {'measure_number': 35,
     'not